# **Using a singular model-structure to predict the data**


## Data Import

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# Models
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
import xgboost as xgb

# Metrics
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split

from utils import *

datasets_folder = './datasets'
figsize = (20,4)
verbosity = 0

metrics = [root_mean_squared_error, r2_score, mean_absolute_error, huber]

tf.config.set_visible_devices([], 'GPU')
print(tf.config.list_physical_devices('GPU'))

[]


In [2]:
pollution_data = read_and_preprocess_dataset(datasets_folder, 'pollution', v=verbosity)
traffic_data = read_and_preprocess_dataset(datasets_folder, 'traffic', v=verbosity, radius=2)
weather_data = read_and_preprocess_dataset(datasets_folder, 'weather', v=verbosity)

stations = list(pollution_data.keys())
agents = list(set(agent for station in stations for agent in pollution_data[station].keys()))

## Data Merging

- Traffic data is per-station, but it should be normalized globally. To do so we first merge the data, we apply normalization, and later use the date and the statino to merge the data with the pollutants.

In [8]:
merged_traffic_data = pd.concat(
    [df.assign(Station=key) for key, df in traffic_data.items()]
)
print(merged_traffic_data.resample('1h').max().isna().sum())
merged_traffic_data

Traffic_value    0
Station          0
dtype: int64


,Traffic_value,Station
Date,,
2019-01-01 00:00:00,27629.0,GIARDINI MARGHERITA
2019-01-01 01:00:00,44430.0,GIARDINI MARGHERITA
2019-01-01 02:00:00,40318.0,GIARDINI MARGHERITA
2019-01-01 03:00:00,25661.0,GIARDINI MARGHERITA
2019-01-01 04:00:00,15906.0,GIARDINI MARGHERITA
...,...,...
2024-12-31 19:00:00,19786.0,VIA CHIARINI
2024-12-31 20:00:00,16239.0,VIA CHIARINI
2024-12-31 21:00:00,7226.0,VIA CHIARINI


In [17]:
scaler = StandardScaler()
scaled_traffic_data = merged_traffic_data.copy()
scaled_traffic_data['Traffic_value'] = scaler.fit_transform(merged_traffic_data[['Traffic_value']])
scaled_traffic_data

,Traffic_value,Station
Date,,
2019-01-01 00:00:00,-0.366692,GIARDINI MARGHERITA
2019-01-01 01:00:00,0.111507,GIARDINI MARGHERITA
2019-01-01 02:00:00,-0.005531,GIARDINI MARGHERITA
2019-01-01 03:00:00,-0.422706,GIARDINI MARGHERITA
2019-01-01 04:00:00,-0.700359,GIARDINI MARGHERITA
...,...,...
2024-12-31 19:00:00,-0.589924,VIA CHIARINI
2024-12-31 20:00:00,-0.690881,VIA CHIARINI
2024-12-31 21:00:00,-0.947414,VIA CHIARINI


- weather data is global, we only have to normalize it.

In [32]:
scaler = StandardScaler()
scaled_weather_data = weather_data.copy()
scaled_weather_data[:] = scaler.fit_transform(scaled_weather_data)
print(scaled_weather_data.resample('1h').max().isna().sum())
scaled_weather_data

TAVG          0
PREC          0
RHAVG         0
RAD           0
W_SCAL_INT    0
W_VEC_DIR     0
LEAFW         0
dtype: int64


,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,
2019-01-01 00:00:00,-1.746206,-0.136824,1.318018,-0.661589,-1.419227,0.045012,-0.465156
2019-01-01 01:00:00,-1.828166,-0.136824,1.382433,-0.661589,-1.419227,0.846389,-0.465156
2019-01-01 02:00:00,-1.781331,-0.136824,1.288289,-0.661589,-1.088148,-0.523708,-0.465156
2019-01-01 03:00:00,-1.816457,-0.136824,1.278379,-0.661589,-1.308868,-0.171911,-0.465156
2019-01-01 04:00:00,-1.746206,-0.136824,1.020722,-0.661589,-1.308868,-0.357363,-0.465156
...,...,...,...,...,...,...,...
2024-12-31 19:00:00,-1.113941,-0.136824,0.743245,-0.661589,-0.260449,0.441767,-0.465156
2024-12-31 20:00:00,-0.985147,-0.136824,0.525228,-0.661589,0.236170,0.355222,-0.465156
2024-12-31 21:00:00,-1.055398,-0.136824,0.629282,-0.661589,0.236170,0.345107,-0.465156


- We can now divide the traffic per station, merge them with the weather and create the input sequences. For every agent of the same station, the input data is the same.
    
    We might also want to consider adding new informations, like the date encodings and so on...

In [29]:
station_data = {
    station: pd.merge(
        scaled_traffic_data[scaled_traffic_data['Station'] == station].drop(columns=['Station']),
        scaled_weather_data,
        left_index=True,
        right_index=True
    )
    for station in stations
}
for station in station_data:
    display(station, station_data[station].resample('1h').max().isna().sum(), station_data[station].head(3))

'GIARDINI MARGHERITA'

Traffic_value    0
TAVG             0
PREC             0
RHAVG            0
RAD              0
W_SCAL_INT       0
W_VEC_DIR        0
LEAFW            0
dtype: int64

,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,
2019-01-01 00:00:00,-0.366692,-1.746206,-0.136824,1.318018,-0.661589,-1.419227,0.045012,-0.465156
2019-01-01 01:00:00,0.111507,-1.828166,-0.136824,1.382433,-0.661589,-1.419227,0.846389,-0.465156
2019-01-01 02:00:00,-0.005531,-1.781331,-0.136824,1.288289,-0.661589,-1.088148,-0.523708,-0.465156


'PORTA SAN FELICE'

Traffic_value    0
TAVG             0
PREC             0
RHAVG            0
RAD              0
W_SCAL_INT       0
W_VEC_DIR        0
LEAFW            0
dtype: int64

,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,
2019-01-01 00:00:00,0.011177,-1.746206,-0.136824,1.318018,-0.661589,-1.419227,0.045012,-0.465156
2019-01-01 01:00:00,0.599669,-1.828166,-0.136824,1.382433,-0.661589,-1.419227,0.846389,-0.465156
2019-01-01 02:00:00,0.316324,-1.781331,-0.136824,1.288289,-0.661589,-1.088148,-0.523708,-0.465156


'VIA CHIARINI'

Traffic_value    0
TAVG             0
PREC             0
RHAVG            0
RAD              0
W_SCAL_INT       0
W_VEC_DIR        0
LEAFW            0
dtype: int64

,Traffic_value,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,LEAFW
Date,,,,,,,,
2019-01-01 00:00:00,-0.913856,-1.746206,-0.136824,1.318018,-0.661589,-1.419227,0.045012,-0.465156
2019-01-01 01:00:00,-0.724523,-1.828166,-0.136824,1.382433,-0.661589,-1.419227,0.846389,-0.465156
2019-01-01 02:00:00,-0.787369,-1.781331,-0.136824,1.288289,-0.661589,-1.088148,-0.523708,-0.465156


Sequences should also be indexed with the date of the last element, so that merging with the agent will be easier. To do so, we simply use one variable for the sequences and one for the dates.

In [89]:
hourly_time_steps = 3
daily_time_steps = 24

In [ ]:
station_sliding_windows = {}
for station in station_data:
    sequences, dates = create_sequences(
        station_data[station],
        pd.Series(station_data[station].index, index=station_data[station].index),
        time_steps=hourly_time_steps
    )
    dates = dates.index.tolist()
    station_sliding_windows[station] = sequences, dates

In [101]:
station_sequences = {}
for station in station_data:
    sequences, dates = create_sequences(
        station_data[station],
        pd.Series(station_data[station].index, index=station_data[station].index),
        time_steps=daily_time_steps,
        sliding_window=False
    ) 
    station_sequences[station] = sequences, dates

- At this point we should create the actual train data composed of sequence + agentID and the target values. We could of course create an huge dataset and directly use it, but I dont know if it is too big and with too many repeated values (sequences are repeated for each agent and each sequence share some rows).

    <!-- For now, I simply try to make a dataset made of `in=(station_date, sequence)`, `x=(station_date, agent)` and `y=(target)`. I don't want the model to learn station and agent.  -->
    The model should be able to take a batch and use the dates to access the sequences, create the embeddings, and use the agent to predict the concentration using the correct classification head. Since that we have to map a dict would be suitable for fast data access.

- Also, at this point we have to divide hourly and daily agents because the embedding we are trying to create for the stations can be quite different. For example if we have to predict an hourly pollutant, we might want to use 3/5 prev. hours. For daily agents instead, the whole 24 hours might be helpful.

In [219]:
def create_key(station, date):
    return (station+' '+str(date)).replace(' ','_')

hourly_input_data = {}
daily_input_data = {}
for station in station_sequences:
    for seq, date in zip(station_sliding_windows[station][0], station_sliding_windows[station][1]):
        hourly_input_data[create_key(station, date)] = seq
    for seq, date in zip(station_sequences[station][0], station_sequences[station][1]):
        daily_input_data[create_key(station, date)] = seq

h_keys = []
h_agentsIDs = []
h_values = []

d_keys = []
d_agentsIDs = []
d_values = []

for station in pollution_data:
    for agent in pollution_data[station]:
        for target, date in zip(pollution_data[station][agent]['Agent_value'], pollution_data[station][agent].index):
            if agent in ('PM10', 'PM2.5'):
                d_keys.append(create_key(station, date))
                d_agentsIDs.append(agents.index(agent))
                d_values.append(target)
            else: 
                h_keys.append(create_key(station, date))
                h_agentsIDs.append(agents.index(agent))
                h_values.append(target)

## Model

- The model will use a TensorFlow DataSet for better something (something's gotta be better right?).

Remove rows if there is no input data

In [220]:
# Filter for hourly data
valid_h_indices = [i for i, k in enumerate(h_keys) if k in hourly_input_data]
h_keys       = [h_keys[i]       for i in valid_h_indices]
h_agentsIDs  = [h_agentsIDs[i]  for i in valid_h_indices]
h_values     = [h_values[i]     for i in valid_h_indices]

# Filter for daily data
valid_d_indices = [i for i, k in enumerate(d_keys) if k in daily_input_data]
d_keys       = [d_keys[i]       for i in valid_d_indices]
d_agentsIDs  = [d_agentsIDs[i]  for i in valid_d_indices]
d_values     = [d_values[i]     for i in valid_d_indices]


In [248]:
def make_tf_loader(data_dict, time_steps, features):
    def _load_sample(key, task_id, y):
        k = key.numpy().decode("utf-8")
        x = data_dict[k].astype("float32")   # [time_steps, features]
        return x, task_id, y

    def _tf_load(key, task_id, y):
        x, t, yy = tf.py_function(
            func=_load_sample,
            inp=[key, task_id, y],
            Tout=[tf.float32, tf.int32, tf.float32]
        )
        # set static shapes so batching works
        x.set_shape([time_steps, features])
        t.set_shape([])
        yy.set_shape([])
        return x, t, yy

    return _tf_load

hourly_input_shape = (list(hourly_input_data.values())[0]).shape
daily_input_shape =  (list(daily_input_data.values())[0]).shape
h_loader = make_tf_loader(hourly_input_data, *hourly_input_shape)
d_loader = make_tf_loader(daily_input_data, *daily_input_shape)

hourly_dataset = tf.data.Dataset.from_tensor_slices((h_keys, h_agentsIDs, h_values))
daily_dataset = tf.data.Dataset.from_tensor_slices((d_keys, d_agentsIDs, d_values))

hourly_dataset = hourly_dataset.map(h_loader, num_parallel_calls=tf.data.AUTOTUNE)
daily_dataset = daily_dataset.map(d_loader, num_parallel_calls=tf.data.AUTOTUNE)

- As we said earlier, the model should be able to use the station and date to access the actual training sequences.

In [249]:
import tensorflow as tf

class MultiAgentModel(tf.keras.Model):
    def __init__(
        self,
        input_shape,        # e.g. (time_steps, features)
        num_outputs,          # integer: how many different heads/tasks
        lstm_units=(128,64),
        head_hidden_units=32,
    ):
        super().__init__()
        self.num_outputs = num_outputs

        # 1) Shared LSTM encoder
        self.encoder = tf.keras.Sequential(name="encoder")
        self.encoder.add(tf.keras.layers.Masking(input_shape=input_shape,
                                                 name="masking"))
        for i, units in enumerate(lstm_units):
            return_seq = (i < len(lstm_units)-1)
            self.encoder.add(tf.keras.layers.LSTM(
                units,
                return_sequences=return_seq,
                name=f"lstm_{i}"
            ))

        # 2) Build all regression heads, then fuse into one big Dense
        #    We'll concatenate them so we end up with a [B, num_outputs] output.
        self.heads = []
        for tid in range(num_outputs):
            head = tf.keras.Sequential(name=f"head_{tid}")
            head.add(tf.keras.layers.InputLayer(input_shape=(lstm_units[-1],)))
            head.add(tf.keras.layers.Dense(head_hidden_units,
                                           activation="relu",
                                           name="dense_hidden"))
            head.add(tf.keras.layers.Dense(1,
                                           activation=None,
                                           name="dense_output"))
            self.heads.append(head)

    def call(self, inputs, training=False):
        x, task_ids = inputs
        # 1) shared embedding
        embs = self.encoder(x, training=training)      # [B, emb_dim]

        # 2) run every head in parallel → list of [B,1]
        out_per_task = [h(embs, training=training) for h in self.heads]
        # 3) concat to [B, num_outputs]
        all_preds = tf.concat(out_per_task, axis=-1)

        # 4) pick each row’s column given task_ids
        #    build indices [[0,tid0],[1,tid1],...]
        batch_idx = tf.range(tf.shape(task_ids)[0])
        idx = tf.stack([batch_idx, task_ids], axis=1)  # [B,2]
        # 5) gather the correct prediction for each sample → [B]
        return tf.gather_nd(all_preds, idx)

    def train_step(self, data):
        x_batch, task_ids, y_batch = data
        with tf.GradientTape() as tape:
            preds = self((x_batch, task_ids), training=True)  # [B]
            loss  = self.compiled_loss(y_batch, preds)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # update metrics (if any)
        self.compiled_metrics.update_state(y_batch, preds)
        return {m.name: m.result() for m in self.metrics}


## Training

In [ ]:
def split_dataset(ds, test_frac=0.15, val_frac=0.1, batch_size=32):
    total_size = len(list(ds))  # you can also use len(h_keys) or h_ds_raw.cardinality()
    test_size  = int(test_frac * total_size)
    val_size   = int(val_frac * (total_size - test_size))  # Val split after test
    train_size = total_size - test_size - val_size

    # Split into test and train
    test_ds = ds.take(test_size)
    train_ds = ds.skip(test_size)

    # Shuffle the train set
    train_ds = train_ds.shuffle(train_size, seed=42)

    # Now split train into train and validation
    val_ds = train_ds.take(val_size)
    train_ds = train_ds.skip(val_size)

    return (
        train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE),
        test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    )


In [251]:
batch_size = 32

h_train, h_val, h_test = split_dataset(hourly_dataset, batch_size=batch_size)
d_train, d_val, d_test = split_dataset(daily_dataset, batch_size=batch_size)

### Daily

In [252]:
d_model = MultiAgentModel(
    input_shape=daily_input_shape,
    num_outputs=len(agents),
    lstm_units=(128, 64),
    head_hidden_units=32,
)

In [253]:
d_model.compile(
    optimizer="adam",
    loss="mse",       # mean squared error for regression
    metrics=["mae"],  # mean absolute error
)

d_hist = d_model.fit(
    x=d_train,
    validation_data=d_val,
    epochs=5
)
plot_history(d_hist)

Epoch 1/5
 61/268 [=====>........................] - ETA: 6s - loss: nan - mae: nan

KeyboardInterrupt: 